# Project1 Classification of Text Data
### Alex Wang UID:905029621
### Zhijie Wang UID:304946569

In [ ]:
import numpy as np
import itertools
import operator
import pandas as pd
import re
import math
import gc
import string
import matplotlib.pyplot as plt; plt.rcdefaults()
import sklearn.linear_model as sk
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing, cross_validation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import text
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_curve,auc
from statsmodels.discrete.discrete_model import Logit
from nltk import SnowballStemmer
from collections import Counter
from collections import defaultdict
#from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
#from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import sklearn.metrics
import nltk
from nltk import pos_tag
import pickle as cPickle

In [ ]:
## Functions
def roc_plot(y_test,y_score,label):
  fpr, tpr, thresholds = roc_curve(y_test, y_score)
  roc_auc = auc(fpr, tpr)

  plt.figure()
  lw = 2
  plt.plot(fpr, tpr, 
           color='darkorange',lw=lw, 
           label=label  % roc_auc)
  plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  plt.xlim([-0.1, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate',fontsize=15)
  plt.ylabel('True Positive Rate',fontsize=15)
  plt.title('Receiver Operating Characteristic (ROC)')
  plt.legend(loc="lower right")
  plt.show()
  return

def stats(true, predicted,classes,title):
    print("==========="+title+"===========")
    print(metrics.classification_report(true, predicted,target_names=classes))
    print("accuracy: " + str(np.mean(predicted == true)))
    return 1
  
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.4f' if normalize else 'd'
    thresh = cm.max() / 4.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cats_eight =['rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey',
             'comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware']
classes = ['Computer technology', 'Recreational activity']
class_comp = ['comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware']
class_rec = ['rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey']
eight_train = fetch_20newsgroups(subset='train', categories=cats_eight, shuffle=True, random_state=42,remove=('headers','footers','quotes'))
eight_test = fetch_20newsgroups(subset='test', categories=cats_eight, shuffle=True, random_state=42,remove=('headers','footers','quotes'))

# Problem (A) Plot Balance of Data

In [ ]:
value_eight = []
value_comp = 0
value_rec = 0
biclass =[]
for category in cats_eight:
    cat = [category]
    subclass = fetch_20newsgroups(subset='train', categories=cat, shuffle=True, random_state=42)
    length_eight = (len(subclass.data))
    value_eight.append(length_eight)
    
for category in class_comp:
    comp_cat = [category]
    comp_subclass = fetch_20newsgroups(subset='train', categories=comp_cat, shuffle=True, random_state=42)
    length_comp = (len(comp_subclass.data))
    value_comp += (length_comp)

biclass.append(value_comp)

for category in class_rec:
    rec_cat = [category]
    rec_subclass = fetch_20newsgroups(subset='train', categories=rec_cat, shuffle=True, random_state=42)
    length_rec = (len(rec_subclass.data))
    value_rec += (length_rec)
biclass.append(value_rec)

In [ ]:
index = np.arange(2)
bar_width = 0.7
bars = plt.barh(index, biclass, bar_width)
plt.xlabel("# Documents")
plt.ylabel('Classes')
plt.title('# Documents/Class')
plt.yticks(index + bar_width/2, ('Computer technology', 'Recreational activity'))
plt.grid(True)
plt.show()

index = np.arange(8)
bar_width = 0.7
bars = plt.barh(index, value_eight, bar_width)
plt.xlabel("# Documents")
plt.ylabel('Subclasses')
plt.title('# Documents/Subclass')
plt.yticks(index + bar_width/2, ('comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 
                                 'comp.sys.mac.hardware', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey'))
plt.grid(True)
plt.show()

# Problem (B) Text Vectorization

In [ ]:
# This is the stemming and lemmatize portion extracted from the TA discussion
wnl = nltk.wordnet.WordNetLemmatizer()
stop_words_skt = text.ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
#nltk.download('stopwords' )
stop_words_en = stopwords.words('english')
combined_stopwords = set.union(set(stop_words_en),set(string.punctuation),set(stop_words_skt))
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

def lemmatize_sent(list_word): 
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]

analyzer = CountVectorizer().build_analyzer()
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))
def stem_rmv_punc(doc):
    return (word for word in lemmatize_sent(analyzer(doc)) if word not in combined_stopwords and not word.isdigit())

In [ ]:
min_dfs = [2,5]
for min_df in min_dfs:
    data_vec = CountVectorizer(min_df=min_df,analyzer=stem_rmv_punc)
    X_train_tfidf = data_vec.fit_transform(eight_train.data)
    print (X_train_tfidf.shape)

In [ ]:
data_vec = CountVectorizer(min_df=2,analyzer=stem_rmv_punc)
#data_vec = CountVectorizer(min_df=5,analyzer=stem_rmv_punc)
train_counts = data_vec.fit_transform(eight_train.data)

tfidf_transformer = TfidfTransformer(smooth_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(train_counts)
#print("Dimensions of TF-IDF vector: "+ str(X_train_tfidf.shape))

# Problem (C) Find the 10 Most Frequenct Terms


In [ ]:
twenty_classes = ['comp.graphics',
                  'comp.os.ms-windows.misc',
                  'comp.sys.ibm.pc.hardware',
                  'comp.sys.mac.hardware',
                  'comp.windows.x',
                  'rec.autos',
                  'rec.motorcycles',
                  'rec.sport.baseball',
                  'rec.sport.hockey',
                  'alt.atheism',
                  'sci.crypt',
                  'sci.electronics',
                  'sci.med',
                  'sci.space',
                  'soc.religion.christian',
                  'misc.forsale',
                  'talk.politics.guns',
                  'talk.politics.mideast',
                  'talk.politics.misc',
                  'talk.religion.misc'
                ]
stop_words = text.ENGLISH_STOP_WORDS
docs_in_single_class = []
# flatten the documents in each category into ONE document in each category
for docClass in twenty_classes:
    class_data = fetch_20newsgroups(subset='train',categories=[docClass]).data
    temp = " ".join(class_data)
    docs_in_single_class.append(temp)
    
class_Vec = CountVectorizer(min_df=2,analyzer=stem_rmv_punc)
#class_Vec = CountVectorizer(min_df=5,analyzer=stem_rmv_punc)
class_Vec_train = class_Vec.fit_transform(docs_in_single_class)
class_Vec_train.shape

In [ ]:

tfidf_transformer = TfidfTransformer(smooth_idf=False)
tficf_class_train = tfidf_transformer.fit_transform(class_Vec_train)
#tficf_class_train_copy = tficf_class_train.copy()
features = class_Vec.get_feature_names()
from scipy.sparse import csr_matrix

for i in [2, 3, 14, 15]:
    row = tficf_class_train.getrow(i).toarray()[0].ravel()
    top_ten_indicies = row.argsort()[::-1][:10]
    print(twenty_classes[i])
    print(''.join('{}, '.format(features[a]) for a in top_ten_indicies))
    print('\n')


# Problem (D) SVD & NMF

In [ ]:
#SVD
svd = TruncatedSVD(n_components=50)
train_lsi = svd.fit_transform(X_train_tfidf)

#NMF
modleNMF = NMF(n_components=50)
train_NMF = modleNMF.fit_transform(X_train_tfidf)

## Construct Train and Test Set

In [ ]:
test_label = (eight_test.target[:]>=4)
train_label = (eight_train.target[:]>=4)

test_counts = data_vec.transform(eight_test.data)
test_tfidf = tfidf_transformer.fit_transform(test_counts)
test_lsi = svd.transform(test_tfidf)
test_NMF = modleNMF.transform(test_tfidf)

# Problem (E) Support Vector Machine Classifier

## Hard-margin SVM

**(1) LSI Method**

In [ ]:
# Training
clf = SVC(C=1000,kernel='linear')
clf.fit(train_lsi, train_label)
predicted_lsi = clf.predict(test_lsi)

In [ ]:
#roc
label = 'ROC curve for SVC   (area = %0.2f)'
predicted_score = clf.decision_function(test_lsi)
roc_plot(test_label,predicted_score,label)

#stats
stats(test_label,predicted_lsi,classes,"SVM")

#confusion matrix
cnf_matrix = confusion_matrix(test_label, predicted_lsi)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
print('\n====================================\n')
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion Matrix, without Normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion Matrix, with Normalization')
plt.show()

**(2) NMF Method**

In [ ]:
# Training
clf = SVC(C=1000,kernel='linear')
clf.fit(train_NMF, train_label)
predicted_NMF = clf.predict(test_NMF)

In [ ]:
#roc
label = 'ROC curve for SVC   (area = %0.2f)'
predicted_score = clf.decision_function(test_NMF)
roc_plot(test_label,predicted_score,label)

#stats
stats(test_label,predicted_NMF,classes,"SVM")

#confusion matrix
cnf_matrix = confusion_matrix(test_label, predicted_NMF)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion matrix, without normalization')
plt.show()

In [ ]:
predicted_lsi=predicted_NMP=[]
gc.collect()

## Soft-margin SVM

In [ ]:
clf = SVC(C=0.001,kernel='linear')
clf.fit(train_lsi, train_label)
predicted_lsi = clf.predict(test_lsi)
classes = ['Computer technology', 'Recreational activity']

In [ ]:
#roc
label = 'ROC curve for SVC   (area = %0.2f)'
predicted_score = clf.decision_function(test_lsi)
roc_plot(test_label,predicted_score,label)

#stats
stats(test_label,predicted_lsi,classes,"SVM")

#confusion matrix
cnf_matrix = confusion_matrix(test_label, predicted_lsi)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion matrix, without normalization')
plt.show()

# Problem(F) Find best gamma

In [ ]:
f1=[]
temp = pd.DataFrame()
gamma_range = [10 ** i for i in range(-3,4)]
scoring = ['f1']
for test_gamma in gamma_range:
    find_SVM = SVC(C=test_gamma,kernel='linear')
    # 5-fold cross-validation
    scores = cross_validate(find_SVM, train_lsi, train_label,
                            scoring=scoring,cv=5,return_train_score=False)
    temp['f1'] = scores['test_f1']
    # record the 5 f1 scores with the same gamma value, and average them
    mean = temp['f1'].mean()
    f1.append(mean)

df = pd.DataFrame()
df['gamma'] = gamma_range
df['f1'] = f1
df

In [ ]:
print('The value of gamma with the highest F1 score is {0}'.format(df['gamma'][df['f1'].idxmax()]))

In [ ]:
find_SVM = SVC(C=1000, kernel='linear')
find_SVM.fit(train_lsi, train_label)
#find_SVM.fit(train_NMF, train_label)
predicted_label = find_SVM.predict(test_lsi)
#predicted_label = find_SVM.predict(test_NMF)
classes = ['Computer technology', 'Recreational activity']

#stats
stats(test_label,predicted_label,classes,"SVM － LSI")
#stats(test_label,predicted_label,classes,"SVM － NMF")

#confusion matrix
cnf_matrix = confusion_matrix(test_label, predicted_label)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=False,
                      title='Confusion matrix with best Gamma')
plt.show()


In [ ]:
# Delete unnecessary data to free memories
del df,temp
acc=pre=rec=[]
gc.collect()

# Problem(G) Naive Bayes Rule

In [ ]:
#Train_lsi negative
#https://stackoverflow.com/questions/24169238/dealing-with-negative-values-in-sklearn-multinomialnb
clf = MultinomialNB()
clf.fit(train_NMF, train_label)
clf_predicted = clf.predict(test_NMF)

#roc
label = 'ROC curve for Naive Bayes   (area = %0.2f)'
predicted_score = clf.predict_proba(test_NMF)
roc_plot(test_label,predicted_score[:,1],label)

#stats
stats(clf_predicted,test_label,classes,"NB")

#confusion matrix
cnf_matrix = confusion_matrix(clf_predicted,test_label)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

# Problem (H) Logistic Regression

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
clf.fit(train_lsi,train_label)
predicted = clf.predict(test_lsi)

#roc
label = 'ROC curve for Linear Regression   (area = %0.2f)'
predicted_score = clf.predict_proba(test_lsi)
roc_plot(test_label,predicted_score[:,1],label)

#stats
stats(predicted,test_label,classes,"Linear Regression")

#confusion matrix
cnf_matrix = confusion_matrix(predicted,test_label)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion matrix, without normalization')
plt.show()

## L1 norm Logistic

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=1000,penalty='l1')
clf.fit(train_NMF,train_label)
predicted = clf.predict(test_NMF)

#roc
label = 'ROC curve for Linear Regression   (area = %0.2f)'
predicted_score = clf.predict_proba(test_NMF)
roc_plot(test_label,predicted_score[:,1],label)

#stats
stats(predicted,test_label,classes,"Linear Regression")

#confusion matrix
cnf_matrix = confusion_matrix(predicted,test_label)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion matrix, without normalization')
plt.show()

## L2 norm logistic

In [ ]:
clf = linear_model.LogisticRegression(C=1000, penalty='l2')
clf.fit(train_lsi,train_label)
predicted = clf.predict(test_lsi)

#roc
label = 'ROC curve for Linear Regression   (area = %0.2f)'
predicted_score = clf.predict_proba(test_lsi)
roc_plot(test_label,predicted_score[:,1],label)

#stats
stats(predicted,test_label,classes,"Linear Regression")

#confusion matrix
cnf_matrix = confusion_matrix(predicted,test_label)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,normalize=True,
                      title='Confusion matrix, without normalization')
plt.show()

# Problem(I) Multiclass Classification

In [ ]:
class_multi = ['comp.sys.ibm.pc.hardware' , 'comp.sys.mac.hardware', 'misc.forsale','soc.religion.christian']
train_multi = fetch_20newsgroups(subset='train',categories=class_multi,shuffle=True,random_state=42)
test_multi = fetch_20newsgroups(subset='test',categories=class_multi,shuffle=True,random_state=42)

train_multi_counts = data_vec.fit_transform(train_multi.data)
train_multi_tfidf = tfidf_transformer.fit_transform(train_multi_counts)
train_multi_lsi = svd.fit_transform(train_multi_tfidf)
train_multi_NMF = modleNMF.fit_transform(train_multi_tfidf)
train_multi_label = (train_multi.target[:]>=4)

test_multi_counts = data_vec.transform(test_multi.data)
test_multi_tfidf = tfidf_transformer.transform(test_multi_counts)
test_multi_lsi = svd.transform(test_multi_tfidf)
test_multi_NMF = modleNMF.transform(test_multi_tfidf)
test_multi_label = (test_multi.target[:]>=4)

## Multi - Naive Bayes

In [ ]:
clf = MultinomialNB().fit(train_multi_NMF,train_multi.target)
predicted = clf.predict(test_multi_NMF)

In [ ]:
#stats
stats(test_multi.target,predicted,class_multi,"Linear Regression")

#confusion matrix
cnf_matrix = confusion_matrix(predicted,test_multi.target)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_multi,normalize=False,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_multi,normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

## Multi - SVM

In [ ]:
clf_ovo = OneVsOneClassifier(LinearSVC(C=1000,random_state=42)).fit(train_multi_lsi,train_multi.target)
predicted = clf_ovo.predict(test_multi_lsi)

#stats
stats(test_multi.target,predicted,class_multi,"One Vs. One Classification")

#confusion matrix
cnf_matrix = confusion_matrix(predicted,test_multi.target)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_multi,normalize=False,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_multi,normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

In [ ]:
clf_ovr = OneVsRestClassifier(LinearSVC(C=100,random_state=42)).fit(train_multi_lsi,train_multi.target)
predicted = clf_ovr.predict(test_multi_lsi)

#stats
stats(test_multi.target,predicted,class_multi,"One Vs. Rest Classification")

#confusion matrix
cnf_matrix = confusion_matrix(predicted,test_multi.target)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_multi,normalize=False,
                      title='Confusion matrix, without normalization')
plt.show()

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_multi,normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()